# Process job adverts and load to DB

This program processes the scraped job ads saved in ./raw/, searches for org.no. in business register, loads the trained neural-net model (plus helper-objects), estimates ISCO08, and saves it to a pandas data-frame. This then gets inserted into a MySQL database.

Loading necessary libraries (and probably some unnecessary ones...)

In [134]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import time
import datetime
from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import html
import re
import os
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import pickle
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
from keras.models import load_model

Define input and output folders for files

In [135]:
in_path = '/home/radbrt/notebooks/hackathon/job_vacancies/raw/'
out_path = '/home/radbrt/notebooks/hackathon/job_vacancies/processed/'
error_path = '/home/radbrt/notebooks/hackathon/job_vacancies/errorfiles/'

Define functions to clean up ad-text (remove stopwords, convert to lowercase, remove numbers etc)

In [136]:
# download stopwords if not done yet
# nltk.download("stopwords")
stops = set(stopwords.words("norwegian"))

def alphanum(text):
    return re.sub("[^A-Za-zæÆøØåÅ]", " ", text)

def remove_stops(text):
    return " ".join([word for word in text.split() if word not in stops])

def clean_text(text):
    cleantext = text.lower()
    cleantext = alphanum(cleantext)
    cleantext = remove_stops(cleantext)
    return cleantext

Define function to search for company name:

In [137]:
def essearch(name):
    try:
        res = es.search(index='er', body={"query": {"multi_match": {"query": name, "fields": ["navn"]}}})
        return res['hits']['hits'][0]['_source']['orgnr']
    except:
        return ''

Load neural-net models, tokenizer and labelencoder:

In [138]:
model = load_model('model.h5')

with open('tokenizr.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
with open('labelencoder.pickle', 'rb') as handle:
    labelencoder = pickle.load(handle)

Function that inputs advert-text, returns predicted ISCO08 code:

In [139]:
def npred(text):
    t_hot = tokenizer.texts_to_matrix([text], mode='binary')
    pc = model.predict_classes(t_hot, batch_size=1)
    isco = labelencoder.inverse_transform(pc)
    return isco[0]

Reading files from /raw and processing them, limit number of files to reduce risk:

In [295]:
files = [file for file in os.listdir(in_path) if file.startswith('jobb')]

In [297]:
len(files)

197

In [298]:
jd = []

Either connect to docker instance of elasticsearch, or separate, permanent instance with real password, saved outside repo. Object names are the same though, comment out %run or the others.

In [299]:
%run ../connect_elastic.py
# es = Elasticsearch(['http://localhost:9200'])

In [300]:
for file in files:
    try:
        e = {}
        f = open(in_path + file, 'r').read()
        funescape = html.unescape(f)
        soup = BeautifulSoup(funescape, 'html.parser')
        arbeidsgiver = soup.find_all('h1', attrs={'id': 'arbeidsgivernavn'})[0].text

        stillingsinfo = soup.find_all('section', 'stillingsinfo')
        stillingsinfo_tittel = stillingsinfo[0].find_all('h1', attrs={'id': 'tittelvalue'})[0].get_text()
        stillingsinfo_text = stillingsinfo[0].find_all('p')[0].get_text()

        detaljer = soup.find_all('table', 'stillingsdetaljer')
        e['title'] = stillingsinfo_tittel
        e['text'] = clean_text(stillingsinfo_text)
        for element in detaljer[0].find_all('td'):
            try:
                h1 = element.find_all('h1')[0].get_text()
                p = element.find_all('p')[0].get_text()
            except:
                h1='X'
                p = 'Y'
            e[h1] = p

        e['orgnr'] = essearch(arbeidsgiver)
        e['isco08'] = str(npred(e['text']))


        jd.append(e)
        os.rename(in_path + file, out_path + file)
    except:
        os.rename(in_path + file, error_path + file)

1/1 [==============================] - 0s


In [301]:
df = pd.DataFrame(jd)

In [302]:
len(df)

197

Convert some column types

In [303]:
def dateconvert(dstring):
    pattern = '\d{2}\.\d{2}\.\d{4}'
    r = re.search(pattern, dstring)
    return datetime.datetime.strptime(r.group(0), '%d.%m.%Y').date()

In [304]:
df.rename(columns={'Antall stillinger': 'num_vacancies',
                  'Registrert': 'valid_from',
                  'Siste publiseringsdato': 'valid_to',
                  }, inplace=True)

df = df[['num_vacancies', 'valid_from', 'valid_to', 'isco08', 'orgnr', 'text', 'title']]


In [305]:
df['valid_from'] = df['valid_from'].apply(dateconvert)
df['valid_to'] = df['valid_to'].apply(dateconvert)

In [306]:
df['num_vacancies'] = pd.to_numeric(df['num_vacancies'], errors='coerce')

Either connect to docker instance of mysql, or separate, permanent instance with real password, saved outside repo. Object names are the same though, comment out %run or the others.

In [307]:
%run ../connect_mysql.py
# engine = create_engine('mysql+mysqlconnector://<user>:<password>@localhost/nav')

Insert to SQL via pandas to_sql method. Failsafe saving to disk with datestamp.

In [308]:
try:
    df.to_sql('vacancies', engine, if_exists='append', index=False)
except:
    df.to_csv('vacancies' + datetime.datetime.now().strftime('%Y%m%d%H%M%S'))

### That's it, folks!